In [ ]:
import jax
from flax import linen as nn
import jax.numpy as jnp
import optax
from flaxmodels import ResNet18
from typing import Callable
import tensorflow
import tqdm

from optimizers import SGD, PAGE
from dataloader import get_cifar10_dataloaders, tf_to_jax

In [ ]:
def init_model(rng: jax.random.PRNGKey, init_batch: tuple[jnp.ndarray, jnp.ndarray]) -> tuple[nn.Module, dict]:
    model = ResNet18(output="logits",
                     pretrained=False,
                     num_classes=10)
    variables = model.init(rng, init_batch[0])
    return model, variables

In [ ]:
def get_cross_entropy_loss_fn(model: nn.Module, train: bool = True) -> Callable:
    def loss_fn(variables: dict, batch: tuple[jnp.ndarray, jnp.ndarray]) -> jnp.ndarray:
        images, labels = batch
        outs  = model.apply(variables,
                            images,
                            mutable=["batch_stats"] if train else None)
        logits, new_batch_stats_dict = outs if train else (outs, {})
        loss = optax.softmax_cross_entropy_with_integer_labels(logits, labels).mean()
        return loss, new_batch_stats_dict.get("batch_stats", {})

    return loss_fn

In [ ]:
def get_compute_accuracy_fn(model: nn.Module) -> Callable:
    def compute_accuracy(variables: dict, batch: tuple[jnp.ndarray, jnp.ndarray]) -> jnp.ndarray:
        images, labels = batch
        logits = model.apply(variables, images, train=False)
        return jnp.mean(logits.argmax(axis=1) == labels)
    return compute_accuracy

In [ ]:
def train(variables: dict,
          optimizer: SGD,
          train_loader,
          val_loader,
          num_epochs: int,
		  compute_accuracy) -> dict:
    state = optimizer.init(variables)
    for epoch in range(num_epochs):
        for batch in tqdm(train_loader):
            jax_batch = tf_to_jax(batch)
            loss, state = optimizer.update(state, jax_batch)
            print(f"loss = {loss}")
        
        val_acc, val_loss, val_count = 0, 0, 0
        for batch in val_loader:
            jax_batch = tf_to_jax(batch)
            acc = compute_accuracy({"params": state.params, "batch_stats": state.batch_stats}, jax_batch)
            val_acc += acc * batch[0].shape[0]
            val_count += batch[0].shape[0]

        print(f"Epoch {epoch + 1}: val_acc={val_acc / val_count}, val_loss={val_loss / val_count}")
    

In [ ]:
rng = jax.random.PRNGKey(42)

BS, BS_HAT = 256, 16


train_loader, val_loader = get_cifar10_dataloaders(batch_size=BS)
init_batch = tf_to_jax(next(iter(train_loader)))
resnet18, variables = init_model(rng, init_batch)

compute_accuracy = jax.jit(get_compute_accuracy_fn(resnet18))

train_loss_fn = get_cross_entropy_loss_fn(resnet18, train=True)
eval_loss_fn = get_cross_entropy_loss_fn(resnet18, train=False)
# optimizer = SGD(train_loss_fn,
#                 lr=1e-4,
#                 need_jit=True)



optimizer = PAGE(train_loss_fn,
                 eval_loss_fn,
                 p=BS_HAT / (BS + BS_HAT),
                 lr=1e-4,
                 bs=BS,
                 bs_hat=BS_HAT,
                 need_jit=True)

train(variables=variables,
      optimizer=optimizer,
      train_loader=train_loader,
      val_loader=val_loader,
      num_epochs=10,
      compute_accuracy=compute_accuracy)